In [1]:
import sys,os
from pandas import Series,DataFrame
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib
#matplotlib.use('agg') 
import matplotlib.pyplot as plt
import openpyxl
from matplotlib.pyplot import cm
from scipy import stats
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
plotlist=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16] # there should be 16 plots in total
dir = 'myc' #you need to have a folder named 'myc' to store the txt files
width=80 #set the width
outdir='results-'+dir+'-rmsd-'+str(width) 
if not os.path.isdir(outdir):
    os.mkdir(outdir) #creat a folder to store the output files
dfs={}
peaklist={}
rmsd_th=1.3
rmsd_average=1
height_th=1.0

In [6]:
for filename in os.listdir(dir):
    if filename.endswith('.txt'):
        print(filename)
        count=1
        dfs[filename]=pd.read_table(dir+'/'+filename,header=None)
        
        dfs[filename].columns=['pos','intensity']
        if count in plotlist:
            print(count)
            fig,ax = plt.subplots(1,1,figsize=(16,6), sharex=True, sharey=False) #only generate one plot in this figure, figure size = 16*6
            ax.set_title(filename[:-4],fontsize=20, fontweight='bold') #set the text formatting of the title and lables
            ax.set_xlabel('position',fontsize=20, fontweight='bold')
            ax.set_ylabel('intensity',fontsize=20, fontweight='bold')
            ax.plot(dfs[filename]['pos'],dfs[filename]['intensity'],color='darkred',lw=2,alpha=1) #plot the original signal
            ax.set_xlim([0,np.max(dfs[filename]['pos'])]) # set the x range of the plot (from 0 to the final position of signal)
            ax.set_ylim([np.min(dfs[filename]['intensity']),1.02*np.max(dfs[filename]['intensity'])]) # set the y range of the plot (from min to 1.02*max)
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'.png',dpi=72,bbox_inches='tight') #save the figure
            plt.close(fig)


        count=2
        window='flat'
        w=np.ones(2*width+1,'d') #generate a rectangle window function with a given width （window = w/w.sum()）
        dfs[filename]['avg']=np.convolve(w/w.sum(),dfs[filename]['intensity'],'same')
        # do the convolution to the original signal with the window function, and get the avg result, 'same' means to keep the same output length as the input signal
        dfs[filename]['sd']=(dfs[filename]['intensity']-dfs[filename]['avg'])**2
        dfs[filename]['msd']=np.convolve(w/w.sum(),dfs[filename]['sd'],'same')
        dfs[filename]['rmsd']=dfs[filename]['msd']**0.5
        # calculate the sd(standard deviation), msd(mean-squared deviation),rmsd(root mean-squared) of the signal

# plot5: the rmsd of the signal (with given window function width)
        if count in plotlist: 
            print(count)
            fig,ax = plt.subplots(1,1,figsize=(16,6), sharex=True, sharey=False)
            ax.set_title(filename[:-4],fontsize=20, fontweight='bold')
            ax.set_xlabel('position',fontsize=20, fontweight='bold')
            ax.set_ylabel('rmsd',fontsize=20, fontweight='bold')
            ax.plot(dfs[filename]['pos'],dfs[filename]['rmsd'],color='black',lw=2,alpha=1)
            ax.set_xlim([0,np.max(dfs[filename]['pos'])])
            ax.set_ylim([0,1.02*np.max(dfs[filename]['rmsd'])])
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+str(width)+'-rmsd.png',dpi=72,bbox_inches='tight')
            plt.close(fig)
        count+=1
        
#plot the distribution of signal intension
        fig,ax = plt.subplots(1,1,figsize=(6,6), sharex=True, sharey=False)
        ax.set_title(filename[:-4],fontsize=20, fontweight='bold')
        ax.set_xlabel('Intensity',fontsize=20, fontweight='bold')
        n, bins, patches = ax.hist(dfs[filename]['intensity'],color='black',bins=100,lw=2,alpha=1,histtype='step')
        #ax.set_xlim([0,np.max(bins)])
        if count in plotlist:
            print(count)
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+str(width)+'-int-hist.png',dpi=72,bbox_inches='tight')
        count+=1
        
        
        
#set the threshold in the plot of signal intension distribution
#the threshold is at the '1/5 point' ('1/5 point' is 1/5 of the max distribution (th*max_val) )
        max_val=np.max(n) # max_val为信号最高值
        th=0.2
        th_=0.1
        th_i=0
        max_i=len(n) # max_i为最高值信号的位置
        done=0
        for i in range(len(n)):
            if done==0:
                if max_val==n[i]:
                    max_i=i 
                if i>max_i: 
                    if max_val*th_>n[i]: #n[i]为当前信号幅值
                        done=1 #检测部分：截取到信号首个小于最高点十分之一的位置（th_ * max_val）
                if max_val*th<n[i]:
                    th_i=i+1   #设置threshold部分：截取到信号最后小于最高点五分之一的位置 （th * max_val）
                
        ax.plot([bins[th_i],bins[th_i]],[0,max_val],color='darkred',lw=1,alpha=1) # plot the threshold
        if count in plotlist:
            print(count)
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+str(width)+'-int-hist_.png',dpi=72,bbox_inches='tight')
        count+=1
        plt.close(fig)
        intensity_level=bins[th_i]
        #intensity_level 为intension distribution五分之一点对应的intensity大小

        
        
        
#plot the distribution of signal RMSD
        fig,ax = plt.subplots(1,1,figsize=(6,6), sharex=True, sharey=False)
        ax.set_title(filename[:-4],fontsize=20, fontweight='bold')
        ax.set_xlabel('rmsd',fontsize=20, fontweight='bold')
        n, bins, patches = ax.hist(dfs[filename]['rmsd'],color='black',bins=100,lw=2,alpha=1,histtype='step')
        ax.set_xlim([0,np.max(bins)])
        if count in plotlist:
            print(count)
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+str(width)+'-rmsd-hist.png',dpi=72,bbox_inches='tight')
        count+=1
        
        
# Set the threshold in the plot of signal RMSD distribution:
# Go through the signals till the first '1/10 point' ('1/10 point' is 1/10 of the max distribution (th_*max_val) )
# Find the last '1/5 point' ('1/5 point' is 1/5 of the max distribution (th*max_val) )
# Set the threshold at the '1/5 point'
# The part right to the threshold is regarded as 'noise part'
        max_val=np.max(n)
        th=0.2 
        th_=0.1
        th_i=0
        max_i=len(n)
        done=0
        for i in range(len(n)):
            if done==0:
                if max_val==n[i]:
                    max_i=i
                if i>max_i:
                    if max_val*th_>n[i]:
                        done=1
                if max_val*th<n[i]:
                    th_i=i+1
        rmsd_level=bins[th_i]

        
        
# Redefine the rmsd_level from the 'noise part'
# Calculate the RMSD mean and RMSD standard deviation in the 'noise part' to get the new rmsd_level value
        #print(rmsd_level)
        if rmsd_average==1:
            print('rmsd avg')
            rmsd_mean=np.mean(dfs[filename]['rmsd'][dfs[filename]['rmsd']<=rmsd_level])
            rmsd_std=np.std(dfs[filename]['rmsd'][dfs[filename]['rmsd']<=rmsd_level])
            rmsd_level=rmsd_mean+rmsd_std*(-2*np.log(th))**0.5
            #print(rmsd_level)

       
    
    
# plot the two thresholds in the signal RMSD distribution
# the first threshold is the rmsd_level
# position of the second threshold =  rmsd_th(1.3) * position of the first threshold
        ax.plot([rmsd_level,rmsd_level],[0,max_val],color='darkred',lw=1,alpha=1)
        if count in plotlist:
            print(count)
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+str(width)+'-rmsd-hist_.png',dpi=72,bbox_inches='tight')
        count+=1
        ax.plot([rmsd_level*rmsd_th,rmsd_level*rmsd_th],[0,max_val],color='darkred',lw=1,ls='--',alpha=1)
        if count in plotlist:
            print(count)
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+str(width)+'-rmsd-hist__.png',dpi=72,bbox_inches='tight')
        count+=1
        plt.close(fig)

# plot: find peaks based on RMSD threshold (the second threhold)
        if count in plotlist:
            print(count)
            fig,ax = plt.subplots(1,1,figsize=(16,6), sharex=True, sharey=False)
            ax.set_title(filename[:-4],fontsize=20, fontweight='bold')
            ax.set_xlabel('position',fontsize=20, fontweight='bold')
            ax.set_ylabel('intensity',fontsize=20, fontweight='bold')
            ax.plot(dfs[filename]['pos'],dfs[filename]['intensity'],color='black',lw=2,alpha=1)
            for idx in dfs[filename]['pos'].index:
                if dfs[filename]['rmsd'].ix[idx]>rmsd_level*rmsd_th:
                    ax.scatter(dfs[filename]['pos'].ix[idx],1.01*np.max(dfs[filename]['intensity']),color='darkred',s=10,alpha=1)
                    # 把peak区间的值改为最高值作为红线
            ax.set_xlim([0,np.max(dfs[filename]['pos'])])
            ax.set_ylim([np.min(dfs[filename]['intensity']),1.02*np.max(dfs[filename]['intensity'])])
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+str(width)+'_.png',dpi=72,bbox_inches='tight')
            plt.close(fig)
        count+=1

# plot: smooth the original signal with a window function 
# hanning window is a sin signal, flat window is a rectangle signal
# do the convolution to the original signal with the window function
        width2=width
        window2='hanning'
        intensity_max=0
        peak_width_threshold=0.25
        if window2 in ['hanning', 'hamming', 'bartlett', 'blackman']:
            w2=eval('np.'+window2+'(2*width2+1)')
        else:
            window2='flat'
            w2=np.ones(2*width2+1,'d')
        dfs[filename]['int-smooth']=np.convolve(w2/w2.sum(),dfs[filename]['intensity'],'same')
        
        
        if count in plotlist:
            print(count)
            fig,ax = plt.subplots(1,1,figsize=(16,6), sharex=True, sharey=False)
            ax.set_title(filename[:-4],fontsize=20, fontweight='bold')
            ax.set_xlabel('position',fontsize=20, fontweight='bold')
            ax.set_ylabel('intensity',fontsize=20, fontweight='bold')
            ax.plot(dfs[filename]['pos'],dfs[filename]['int-smooth'],color='black',lw=2,alpha=1)
            ax.set_xlim([0,np.max(dfs[filename]['pos'])])
            ax.set_ylim([np.min(dfs[filename]['intensity']),1.02*np.max(dfs[filename]['intensity'])])
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+window+str(width)+'-int-smooth.png',dpi=72,bbox_inches='tight')
            plt.close(fig)
        count+=1
        
        
# plot10: derivative of smoothed signal
# calculate the derivative of smoothed signal, find both the max and min points
# do the second derivative, to seperate the max points(second derivative<0) and min points (second derivative>0)
        dfs[filename]['int-diff']=dfs[filename]['int-smooth'].diff()
        # 'int-diff' is the first derivative of the smoothed signal
        if count in plotlist:
            print(count)
            fig,ax = plt.subplots(1,1,figsize=(16,6), sharex=True, sharey=False)
            ax.set_title(filename[:-4],fontsize=20, fontweight='bold')
            ax.set_xlabel('position',fontsize=20, fontweight='bold')
            ax.set_ylabel('intensity',fontsize=20, fontweight='bold')
            ax.plot(dfs[filename]['pos'],dfs[filename]['int-diff'],color='black',lw=2,alpha=1)
            ax.set_xlim([0,np.max(dfs[filename]['pos'])])
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+window+str(width)+'-int-diff.png',dpi=72,bbox_inches='tight')
            plt.close(fig)
        count+=1
        
        dfs[filename]['int-diff-bin']=0
        dfs[filename]['int-diff-bin'][dfs[filename]['int-diff']>0]=1
        # if the signal point has a positive derivative, 'int-diff' = 1;
        # if the signal point has a negative derivative, 'int-diff' = 0.
        
        dfs[filename]['int-diff-bin-diff']=dfs[filename]['int-diff-bin'].diff()
        # 'int-diff-bin-diff' is the second derivative result
        
        dfs[filename]['int-diff-bin-diff-max']=dfs[filename]['int-diff-bin-diff']
        dfs[filename]['int-diff-bin-diff-min']=dfs[filename]['int-diff-bin-diff']
        dfs[filename]['int-diff-bin-diff-max'][dfs[filename]['int-diff-bin-diff']>0]=0
        dfs[filename]['int-diff-bin-diff-min'][dfs[filename]['int-diff-bin-diff']<0]=0
        # if the signal point is the max point, 'int-diff-bin-diff' = -1.
        # if the signal point is the min point, 'int-diff-bin-diff' = 1.
        # otherwise, 'int-diff-bin-diff' = 0
        # 'int-diff-bin-diff-max' and 'int-diff-bin-diff-min' has the same information as 'int-diff-bin-diff' but in different format
        
    
        
# plot: find the max peak in original signal  (for the point 'int-diff-bin-diff-max' = -1)  
        if count in plotlist:
            print(count)
            fig,ax = plt.subplots(1,1,figsize=(16,6), sharex=True, sharey=False)
            ax.set_title(filename[:-4],fontsize=20, fontweight='bold')
            ax.set_xlabel('position',fontsize=20, fontweight='bold')
            ax.set_ylabel('intensity',fontsize=20, fontweight='bold')
            ax.plot(dfs[filename]['pos'],dfs[filename]['intensity'],color='darkred',lw=2,alpha=1)
            ax.plot(dfs[filename]['pos'],(-dfs[filename]['int-diff-bin-diff-max'])*1.02*(np.max(dfs[filename]['intensity'])-np.min(dfs[filename]['intensity']))+np.min(dfs[filename]['intensity']),color='black',lw=1,alpha=1)
            ax.set_xlim([0,np.max(dfs[filename]['pos'])])
            ax.set_ylim([np.min(dfs[filename]['intensity']),1.02*np.max(dfs[filename]['intensity'])])
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+window+str(width)+'-int-diff-bin-diff-max.png',dpi=72,bbox_inches='tight')
            plt.close(fig)
        count+=1
        
        
        
# plot: find the min nadir in original signal (for the point 'int-diff-bin-diff-min' = 1)  
        if count in plotlist:
            print(count)
            fig,ax = plt.subplots(1,1,figsize=(16,6), sharex=True, sharey=False)
            ax.set_title(filename[:-4],fontsize=20, fontweight='bold')
            ax.set_xlabel('position',fontsize=20, fontweight='bold')
            ax.set_ylabel('intensity',fontsize=20, fontweight='bold')
            ax.plot(dfs[filename]['pos'],dfs[filename]['intensity'],color='darkred',lw=2,alpha=1)
            ax.plot(dfs[filename]['pos'],(dfs[filename]['int-diff-bin-diff-min'])*1.02*(np.max(dfs[filename]['intensity'])-np.min(dfs[filename]['intensity']))+np.min(dfs[filename]['intensity']),color='black',lw=1,alpha=1)
            ax.set_xlim([0,np.max(dfs[filename]['pos'])])
            ax.set_ylim([np.min(dfs[filename]['intensity']),1.02*np.max(dfs[filename]['intensity'])])
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+window+str(width)+'-int-diff-bin-diff-min.png',dpi=72,bbox_inches='tight')
            plt.close(fig)
        count+=1
        
        
# generate the text file
# pick out the max points (for the point 'int-diff-bin-diff' = -1)
        peaklist[filename]=[]
        for idx in dfs[filename].index:
            # idx 为signal的index
            idx_=idx # idx_用来记录peak signal的index
            if dfs[filename]['int-diff-bin-diff'].ix[idx]==-1: 
            # pick out the max points (for the point 'int-diff-bin-diff' = -1)
            # 此时idx_为peak signal的index
                peak_pos=dfs[filename]['pos'].ix[idx] # peak_pos is the position of the peak
                peak_height=dfs[filename]['intensity'].ix[idx] # peak_height is the intensity of the peak
                signal_to_noise=0
                done=0
                for i in range(width2): #width2 = width = 160
                    if idx-i in dfs[filename].index:
                        if dfs[filename]['intensity'].ix[idx-i]<peak_height/2.0:
                            done=1
                        if dfs[filename]['int-diff-bin-diff'].ix[idx-i]==1:
                            done=1 
                        #检测部分：检测范围向左宽度不超过width(160)；从峰值位置开始向左检测，到峰值1/2的位置；如果检测时候遇到最低点，停止检测
                        if done==0:
                            if peak_height<dfs[filename]['intensity'].ix[idx-i]:
                                peak_height=dfs[filename]['intensity'].ix[idx-i]
                                peak_pos=dfs[filename]['pos'].ix[idx-i]
                                idx_=idx-i
                        #检测期间，更新peak_pos和peak_height
                
                #向右检测
                done=0
                for i in range(width2):
                    if idx+i in dfs[filename].index:
                        if dfs[filename]['intensity'].ix[idx+i]<peak_height/2.0:
                            done=1
                        if dfs[filename]['int-diff-bin-diff'].ix[idx+i]==1:
                            done=1
                        if done==0:
                            if peak_height<dfs[filename]['intensity'].ix[idx+i]:
                                peak_height=dfs[filename]['intensity'].ix[idx+i]
                                peak_pos=dfs[filename]['pos'].ix[idx+i]
                                idx_=idx+i
                
                
                
                #找到向左第一个最低点（检测范围：width*5(160*5)）
                i_min_minus=0
                done=0
                for i in range(width2*5):
                    if idx_-i in dfs[filename].index:
                        if dfs[filename]['int-diff-bin-diff'].ix[idx_-i]==1:
                            done=1
                        if done==0:
                            i_min_minus=i #i_min_minus为峰值向左到最低点的距离
                            
                            
                #找到向右第一个最低点（检测范围：width*5(160*5)）
                i_min_plus=0
                done=0
                for i in range(width2*5):
                    if idx_+i in dfs[filename].index:
                        if dfs[filename]['int-diff-bin-diff'].ix[idx_+i]==1:
                            done=1
                        if done==0:
                            i_min_plus=i #i_min_plus为峰值向右到最低点的距离
                            
    
                local_bgr=(dfs[filename]['intensity'].ix[idx_-i_min_minus]+dfs[filename]['intensity'].ix[idx_+i_min_plus])/2.0
                # local_bgr为峰值左右最低点幅值的平均值
                
                peak_minus=-1
                i_peak_minus=0
                done=0
                for i in range(i_min_minus): 
                #检测范围：峰值到左最低点的部分，峰值向左截取到‘peak_height to local_bgr’四分之一的位置
                #该部分定义为“峰”
                    if idx_-i in dfs[filename].index:
                        if (dfs[filename]['intensity'].ix[idx_-i]-local_bgr)<(peak_height-local_bgr)*peak_width_threshold:
                            #peak_width_threshold=0.25
                            done=1
                        if done==0:
                            peak_minus=dfs[filename]['pos'].ix[idx_-i] #“峰”左边缘的position
                            i_peak_minus=i #“峰”的最高点到左边缘的距离
                            
                #向右检测
                peak_plus=-1
                i_peak_plus=0
                done=0
                for i in range(i_min_plus):
                    if idx_+i in dfs[filename].index:
                        if (dfs[filename]['intensity'].ix[idx_+i]-local_bgr)<(peak_height-local_bgr)*peak_width_threshold:
                            done=1
                        if done==0:
                            peak_plus=dfs[filename]['pos'].ix[idx_+i]
                            i_peak_plus=i
                            

                avg=dfs[filename]['intensity'].ix[idx_]
                # avg用来记录'峰'的总信号intensity
                avg_count=1
                # '峰'的信号数量
                for i in range(i_peak_minus):
                    if idx_-1-i in dfs[filename].index:
                        avg+=dfs[filename]['intensity'].ix[idx_-1-i]
                        avg_count+=1
                for i in range(i_peak_plus):
                    if idx_+1+i in dfs[filename].index:
                        avg+=dfs[filename]['intensity'].ix[idx_+1+i]
                        avg_count+=1
                avg/=avg_count
                # avg为'峰'的平均信号intensity
                
                rmsd=(dfs[filename]['intensity'].ix[idx_]-avg)**2
                rmsd_count=1
                for i in range(i_peak_minus):
                    if idx_-1-i in dfs[filename].index:
                        rmsd+=(dfs[filename]['intensity'].ix[idx_-1-i]-avg)**2
                        rmsd_count+=1
                for i in range(i_peak_plus):
                    if idx_+1+i in dfs[filename].index:
                        rmsd+=(dfs[filename]['intensity'].ix[idx_+1+i]-avg)**2
                        rmsd_count+=1
                rmsd/=rmsd_count
                rmsd=rmsd**0.5
                #rmsd为’峰‘的平均RMSD
                signal_to_noise=rmsd/rmsd_level
                #signal_to_noise为'峰'对于noise的RMSD幅值倍数
                if 30<peak_pos and peak_pos<6020:
                    peaklist[filename].append((peak_pos,peak_minus,peak_plus,peak_plus-peak_minus,peak_height,peak_height-intensity_level,avg,local_bgr,signal_to_noise))
        f=open(outdir+'/'+filename[:-4]+'-'+window+str(width)+'-peaklist.text','w')
        f.write('pos\twhm-\twhm+\tdelta\theight\theight_norm\tavg\tbgr\tsig_to_noise\n')
        for (pos,minus,plus,delta,height,height_norm,avg,bgr,sig_to_noise) in peaklist[filename]:
            f.write(str(pos)+'\t'+str(minus)+'\t'+str(plus)+'\t'+str(delta)+'\t'+str(height)+'\t'+str(height_norm)+'\t'+str(avg)+'\t'+str(bgr)+'\t'+str(sig_to_noise)+'\n')
        # pos = peak_pos, the position of the peak(max point)
        # minus = peak_minus, '峰'左边缘的position
        # plus = peak_plus, '峰'右边缘的position
        # delta = peak_plus-peak_minus, '峰'的range(左右)
        # height = peak_height, max point 的intensity
        # height_norm = peak_height-intensity_level, max point的intensity减去intensity_level(intension distribution五分之一点对应的intensity大小)
        # avg = avg, '峰'的平均信号intensity
        # bgr = local_bgr, 峰值左右最低点幅值的平均值
        # sig_to_noise = signal_to_noise, '峰'对于noise的RMSD幅值倍数
        f.close()

#########################################################################################################
        for (pos,minus,plus,delta,height,height_norm,avg,bgr,sig_to_noise) in peaklist[filename]:
            if sig_to_noise>=rmsd_th and height_norm>height_th:
                    print('peak position: ', pos)
                    print('peak range: ', delta)
                    print('peak height: ', height)
                    print('peak average: ', avg)
                    print('signal to noise: ',sig_to_noise)
#########################################################################################################        

# plot13: find the peaks in the original signal 
# darkblue point: the peak signal point        
        fig,ax = plt.subplots(1,1,figsize=(16,6), sharex=True, sharey=False)
        ax.set_title(filename[:-4],fontsize=20, fontweight='bold')
        ax.set_xlabel('position',fontsize=20, fontweight='bold')
        ax.set_ylabel('intensity',fontsize=20, fontweight='bold')
        ax.plot(dfs[filename]['pos'],dfs[filename]['intensity'],color='darkred',lw=2,alpha=1)
        for (pos,minus,plus,delta,height,height_norm,avg,bgr,sig_to_noise) in peaklist[filename]:
            if sig_to_noise>=rmsd_th and height_norm>height_th:
                ax.scatter([pos],[1.01*height],facecolor='blue',edgecolor='darkblue',s=60,lw=2,alpha=1)
        ax.set_xlim([0,np.max(dfs[filename]['pos'])])
        ax.set_ylim([np.min(dfs[filename]['intensity']),1.02*np.max(dfs[filename]['intensity'])])
        if count in plotlist:
            print(count)
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+window+str(width)+'-peaks.png',dpi=72,bbox_inches='tight')        
        count+=1

        
                
        
# darkblue line: darkblue line: peak width
# the signal part above the darkblue line is regarded as 'peak'
# the height of the darkblue line = peak_width_threshold*(height-bgr)+bgr
# peak_width_threshold = 0.25
# height = peak_height, the intensity of the max point
# bgr = local_bgr, the average intensity of the two min points close to 'peak'
        for (pos,minus,plus,delta,height,height_norm,avg,bgr,sig_to_noise) in peaklist[filename]:
            if sig_to_noise>=rmsd_th and height_norm>height_th:
                ax.plot([minus,plus],[peak_width_threshold*(height-bgr)+bgr,peak_width_threshold*(height-bgr)+bgr],color='darkblue',lw=2,alpha=1)      
        if count in plotlist:
            print(count)
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+window+str(width)+'-peaks-widths.png',dpi=72,bbox_inches='tight')
        count+=1

# gray line: bgr
# bgr = local_bgr, 峰值左右最低点幅值的平均值
        for (pos,minus,plus,delta,height,height_norm,avg,bgr,sig_to_noise) in peaklist[filename]:
            if sig_to_noise>=rmsd_th and height_norm>height_th:
                ax.plot([minus,plus],[bgr,bgr],color='gray',lw=1,alpha=1)
        if count in plotlist:
            print(count)
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+window+str(width)+'-peaks-widths-bgr.png',dpi=72,bbox_inches='tight')
            plt.close(fig)
        count+=1

# black line: avg
# avg = avg, '峰'的平均信号intensity
        for (pos,minus,plus,delta,height,height_norm,avg,bgr,sig_to_noise) in peaklist[filename]:
            if sig_to_noise>=rmsd_th and height_norm>height_th:
                ax.plot([minus,plus],[avg,avg],color='black',lw=1,alpha=1)
        if count in plotlist:
            print(count)
            fig.savefig(outdir+'/'+filename[:-4]+'.'+str(count)+'.'+'-'+window+str(width)+'-peaks-widths-bgr-avg.png',dpi=72,bbox_inches='tight')
            plt.close(fig)
        count+=1

MYC-human_HepG2_ENCFF267YIK.txt
1
2
3
4
5
rmsd avg
6
7
8


/Users/zhangchaojie/miniconda3/envs/2019_bio/lib/python3.6/site-packages/ipykernel_launcher.py:161: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
/Users/zhangchaojie/miniconda3/envs/2019_bio/lib/python3.6/site-packages/ipykernel_launcher.py:162: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


9
10
11
12


/Users/zhangchaojie/miniconda3/envs/2019_bio/lib/python3.6/site-packages/ipykernel_launcher.py:274: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
/Users/zhangchaojie/miniconda3/envs/2019_bio/lib/python3.6/site-packages/ipykernel_launcher.py:277: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
/Users/zhangchaojie/miniconda3/envs/2019_bio/lib/python3.6/site-packages/ipykernel_launcher.py:278: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
/Use

peak position:  460
peak range:  155
peak height:  3.51724137931034
peak average:  2.2200002763041575
signal to noise:  1.494916231609984
peak position:  5920
peak range:  238
peak height:  2.9127155172413794
peak average:  1.393197229836964
signal to noise:  1.830876470249571
13
14
15
16
MYC-human_K562_ENCFF000YLA.txt
1
2
3
4
5
rmsd avg
6
7
8
9
10
11
12
peak position:  449
peak range:  208
peak height:  6.19137645107794
peak average:  3.9091226483213877
signal to noise:  4.906015091566854
peak position:  5967
peak range:  63
peak height:  1.50878938640133
peak average:  1.029208126036484
signal to noise:  1.398039330381475
13
14
15
16
MYC-human_MCF-7_ENCFF000RZJ.txt
1
2
3
4
5
rmsd avg
6
7
8
9
10
11
12
peak position:  141
peak range:  225
peak height:  72.35428571428571
peak average:  50.786396965866004
signal to noise:  26.033293476696283
peak position:  424
peak range:  213
peak height:  47.67428571428571
peak average:  38.13530040053404
signal to noise:  10.772619294529814
peak posi

In [ ]:
dfs.keys()

In [ ]:
dfs['MYC-human_HepG2_ENCFF267YIK.txt']